<h1>
<center>
Module 3 - Naive Bayes
</center>
</h1>
<div class=h1_cell>
<p>

We will look at a competitor to K-NN this week, Naive Bayes. We will continue to work with the tweets so let's bring them in.

</div>

##Use a url

You can see I am using a url below. Reminder of what to do to get it.

1. I went into Google Drive and found the file `tweets_shuffled.csv`. This is the file I wrote out at end of module 2.
2. Right click and then open in Sheets.
3. Publish to web from File menu.
4. Choose csv as the type. Click publish.
5. Copy the url you are given and paste it in below.

Pretty slick. We can now read the file from anywhere with just the url. No need to mount anything.

In [0]:
import pandas as pd
url = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vS1Wbl-BAGw9WxwHU4IA6NSwH8hYF7nKs906aTqsOpV_Vy82R5_shV2W1y0gQVk3cGQJHzCNdGibcPS/pub?output=csv'
tweet_table = pd.read_csv(url)

In [0]:
tweet_table.head()

,label,tweet,length,#,!,a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p,q,r,s,t,u,v,w,x,y,z
0,0,happy weekend! ð#quoteoftheday #quote #mind...,71,5,1,2,1,0,3,9,2,0,2,1,1,1,1,3,3,3,2,2,1,4,4,5,0,1,0,2,0
1,0,i have 1 homegirl ðð,26,0,0,1,0,0,0,2,0,1,2,2,0,0,1,1,0,1,0,0,1,0,0,0,1,0,0,0,0
2,0,@user pray for my country. 2days flooding. g...,96,4,0,11,0,3,2,3,4,3,3,4,0,0,1,1,5,4,2,0,6,5,3,3,0,2,0,4,0
3,0,can't believe i was up through the 3hrs of #or...,136,2,0,4,1,2,2,11,2,3,6,4,0,0,5,0,10,10,2,0,6,6,11,3,2,6,0,1,0
4,0,great way to finish a great week off. #workupa...,102,6,0,7,0,2,0,11,6,2,4,6,0,2,3,1,2,4,1,0,4,7,8,2,0,4,0,3,0


In [0]:
#flush the old directory
!rm -r  'w20_ds_library'

In [0]:
my_github_name = 'FutureDeus'  #replace with your account name

In [0]:
clone_url = f'https://github.com/{my_github_name}/w20_ds_library.git'

In [0]:
#get the latest.
!git clone $clone_url 


Cloning into 'w20_ds_library'...
remote: Enumerating objects: 26, done.
remote: Counting objects: 100% (26/26), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 26 (delta 14), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (26/26), done.


In [0]:
from w20_ds_library import *

In [0]:
%whos

Variable                    Type            Data/Info
-----------------------------------------------------
Callable                    CallableMeta    typing.Callable
TypeVar                     TypingMeta      typing.TypeVar
char_set                    str             #!abcdefghijklmnopqrstuvwxyz
clone_url                   str             https://github.com/FutureDeus/w20_ds_library.git
cm_accuracy                 function        <function cm_accuracy at 0x7f5aa39296a8>
cosine_similarity           function        <function cosine_similarity at 0x7f5aa3929730>
dframe                      TypeVar         ~pd.core.frame.DataFrame
hello_ds                    function        <function hello_ds at 0x7f5aa3929400>
inverse_cosine_similarity   function        <function inverse_cosine_<...>larity at 0x7f5aa39297b8>
knn                         function        <function knn at 0x7f5aa3929598>
knn_tester                  function        <function knn_tester at 0x7f5aa3929620>
my_github_name     

<h2>
Another method: Naive Bayes
</h2>

The problem is still the same. Given a tweet, predict whether it is hate or nohate. The twist is that I am going to reform the question in terms of probabilities using a method called *Naive Bayes*. I'll tell you what is naive about it later. Fow now, you need to do a perspective shift. We will no longer be using a table-based approach that looks at columns of features. Instead we will use counts of things. First, here is the formula we want to compute.

<img src='https://www.dropbox.com/s/gstzvvtvh9b39o8/bayes.png?raw=1'>

 This formula applies for any tweet we are looking at. It produces a prediction for that tweet.
We read the left hand side as "The probability of value O (hate or nohate) given the evidence which we supply from the tweet. In later modules we will be looking at words as evidence. In this module, to keep it simple, we will look at the letters as evidence.
More specifically, on the left hand side looking at P(O|...), the O stands for one of the prediction values. In our case 0 (nohate) or 1 (hate).
On the right hand side of the bar, E1 ... En stands for the evidence we have for making prediction O, i.e., the *unique* letters that show up in the tweet.  So E1 could be 'a', E2 could be 'c', etc. This means we see an 'a' and a 'c' and other letters in the tweet. The way we will use this formula is as follows:
<p>
<ol>
<p>
<li>We have a tweet T made up of a set of letters L. We want to know whether to mark it as hate (1) or nohate (0).</li>
<p>
<li>We compute the left hand side mulitple times, one for each value of O. So in our case we first compute P(nohate|L). What is the probability that T is nohate given its set of letters L. 
<p><li>We then compute P(hate|L). What is the probability that T is hate given its set of letters L.</li>
<p><li>We compare the 2 probabilities. The higher probability wins and we use the associated value for prediction.</li>
</ol>
<p>
In summary, we always compute the probability for all possible prediction values. We only have two, hate and nohate, so we always compute two. If you had three possible values (next week), you would compute three probabilities, etc.
  <p>
Ok, what about the right hand side? We can assume that O has been assigned to one of the classes, nohate or hate, on the left hand side. For sake of discussion, let's say O is assigned to hate. So we are trying to compute P(hate|...).
<ul>
<li>Let's look at the numerator first. We see a sequence of multiplications with terms that look like P(Ei|hate) - I filled in the value for O that we chose. Let's assume E1 is evidence that 'a' is in the set L. Then P('a'|hate) stands for the probability we see the letter 'a' associated with the case hate. This ends up being the total number of times we see 'a' with hate (across all tweets in the training set) divided by the total number of hate tweets (in the training set). If the letter 'a' appears in every hate tweet, then P('a'|hate) = 1.
If it never appears in a hate tweet, then P('a'|hate) is 0. Typically, it is somewhere inbetween.
<li>So we compute the numerator by doing a bunch of lookups for each letter in L in terms of hate. And multiplying together. That is the product of all those Ei terms.</li>
<p><li>What about P(O) at the end? In our example, P(hate). This is just the percentage of tweets that are case hate. Note we can compute P(hate) and P(nohate) just once; they do not change with the tweet. They are global values.</li>
<p><li>We are going to ignore the denominator! Wait. You can't just throw out the denominator. But we can. Do you see why? All we care about is the relative difference between the probabilities for P(hate) and P(nohate). Dividing by a constant does nothing to change that relative difference. If one is greater than another, then dividing both by a constant does not change that. So we throw out the denominator.
</ul>

Ok, so have to get lots of counts of things and do some dividing to get probabilities. I could use a pandas table again to organize our counts. But there is a much faster way: use a dictionary. I'll show you a start and then ask you to complete.


##Get first tweet and create set L

In [0]:
tweet0 = tweet_table.loc[0, 'tweet']
label0 = tweet_table.loc[0, 'label']

In [0]:
label0

0

In [0]:
L0 = set(tweet0).intersection(set('abcdefghijklmnopqrstuvwxyz!#'))
L0

{'!',
 '#',
 'a',
 'b',
 'd',
 'e',
 'f',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'w',
 'y'}

Notice I am ignoring how many times a letter appears in a tweet. Just interested in if it appears. L0 has the letters that appear but not how many times.

# Challenge 1

Build a dictionary of following form:
<pre>
{a_letter: [nohate_count, hate_count], ...}
</pre>

So go through each character in L0 and add it to dictionary with a value of (1,0). Why (1,0)? Because it is a nohate tweet so we add 1 to nohate position.  Here are my results:
<pre>
{'!': [1, 0],
 '#': [1, 0],
 'a': [1, 0],
 'b': [1, 0],
 'd': [1, 0],
 'e': [1, 0],
 'f': [1, 0],
 'h': [1, 0],
 'i': [1, 0],
 'j': [1, 0],
 'k': [1, 0],
 'l': [1, 0],
 'm': [1, 0],
 'n': [1, 0],
 'o': [1, 0],
 'p': [1, 0],
 'q': [1, 0],
 'r': [1, 0],
 's': [1, 0],
 't': [1, 0],
 'u': [1, 0],
 'w': [1, 0],
 'y': [1, 0]}
</pre>

In [0]:
#your code here
char_bag = {key: [1, 0] for key in L0}


In [0]:
char_bag

{'!': [1, 0],
 '#': [1, 0],
 'a': [1, 0],
 'b': [1, 0],
 'd': [1, 0],
 'e': [1, 0],
 'f': [1, 0],
 'h': [1, 0],
 'i': [1, 0],
 'j': [1, 0],
 'k': [1, 0],
 'l': [1, 0],
 'm': [1, 0],
 'n': [1, 0],
 'o': [1, 0],
 'p': [1, 0],
 'q': [1, 0],
 'r': [1, 0],
 's': [1, 0],
 't': [1, 0],
 'u': [1, 0],
 'w': [1, 0],
 'y': [1, 0]}

# Challenge 2

Ok, the harder part. I want char_bag to reflect the char counts for all the tweets in a table. I'm going to do a more traditional split of tweet_table into a training_table and a testing_table. Remember the table got randomly shuffled last week so this is a random selection of testing and training data.

In [0]:
len(tweet_table)/3  #I am carving off 33% to act as testing, rest is training

9836.333333333334

In [0]:
testing_table = tweet_table[:9836]  #use 1/3 to test
training_table = tweet_table[9836:]  #use 2/3 to train

Go ahead and go through every tweet in training_table, compute L, then add counts to appropriate value in pair. My results:
<pre>
{'!': [4469, 214],
 '#': [13176, 956],
 'a': [17757, 1319],
 'b': [11084, 902],
 'c': [12735, 1126],
 'd': [15337, 1121],
 'e': [18088, 1336],
 'f': [12660, 881],
 'g': [13357, 987],
 'h': [15808, 1218],
 'i': [17429, 1312],
 'j': [2585, 241],
 'k': [8812, 663],
 'l': [15773, 1216],
 'm': [14187, 1136],
 'n': [17087, 1287],
 'o': [17550, 1312],
 'p': [11848, 998],
 'q': [747, 68],
 'r': [17356, 1304],
 's': [17670, 1327],
 't': [17430, 1308],
 'u': [15675, 1238],
 'v': [9061, 624],
 'w': [11912, 939],
 'x': [2104, 193],
 'y': [14627, 1001],
 'z': [1255, 105]}
 </pre>

In [0]:
%%time
char_bag = {}

#your code here
char_bag = {key: list([0, 0]) for key in set(char_set)}
for index, row in training_table.iterrows():
  flag = row['label']
  tweet_temp = set(row['tweet']).intersection(set(char_set))
  for i in tweet_temp:
    char_bag[i][flag] += 1

CPU times: user 2.5 s, sys: 8.6 ms, total: 2.5 s
Wall time: 2.52 s


In [0]:
char_bag

{'!': [4469, 214],
 '#': [13176, 956],
 'a': [17757, 1319],
 'b': [11084, 902],
 'c': [12735, 1126],
 'd': [15337, 1121],
 'e': [18088, 1336],
 'f': [12660, 881],
 'g': [13357, 987],
 'h': [15808, 1218],
 'i': [17429, 1312],
 'j': [2585, 241],
 'k': [8812, 663],
 'l': [15773, 1216],
 'm': [14187, 1136],
 'n': [17087, 1287],
 'o': [17550, 1312],
 'p': [11848, 998],
 'q': [747, 68],
 'r': [17356, 1304],
 's': [17670, 1327],
 't': [17430, 1308],
 'u': [15675, 1238],
 'v': [9061, 624],
 'w': [11912, 939],
 'x': [2104, 193],
 'y': [14627, 1001],
 'z': [1255, 105]}

###Have most of what we need

Check it out. If I want to compute something like P('w'|hate), then I look up the value of 'w' which is [11912, 939]. Because it is hate, I pull out 939. That is the numerator. The denominator is the total number of hate tweets in training_table. Let's get that now.

In [0]:
label_list = training_table['label'].tolist()  #convert column into a list of values

In [0]:
hate_count = label_list.count(1)  #only need to compute this once - it does not change
hate_count

1345

In [0]:
nohate_count = label_list.count(0)    #only need to compute this once - it does not change
nohate_count

18328

So P('w'|hate) = 939/1345.

In [0]:
p_w_hate = 939/1345
p_w_hate

0.6981412639405205

Cool. We have a way to compute P(Ei|O). We are still missing P(O). Easy peasy.

In [0]:
p_hate = hate_count/len(training_table)
p_hate

0.0683678137548925

In [0]:
p_nohate = nohate_count/len(training_table)
p_nohate

0.9316321862451075

# Challenge 3

Now for testing. Let's start simple. Give me the probability of P(hate|LT0) and P(nohate|LT0) where LT0 is the char set in the first tweet of the testing_table (not the training_table).

In [0]:
tweet0 = testing_table.loc[0,'tweet']
label0 = testing_table.loc[0,'label']

In [0]:
tweet0

'happy weekend! ð\x9f\x93\x8d#quoteoftheday #quote #mindfulness   #summer #justbe'

In [0]:
label0

0

In [0]:
LT0 = set(tweet0).intersection(char_set)
LT0

{'!',
 '#',
 'a',
 'b',
 'd',
 'e',
 'f',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'w',
 'y'}

In [0]:
#your code here to compute nohate probability

no_numerator = np.multiply(np.prod([np.divide(char_bag[key][0], nohate_count) for key in LT0]), p_nohate)


In [0]:
no_numerator  #1.832109487333443e-05

1.832109487333442e-05

In [0]:
#your code here to compute hate probability

h_numerator = np.multiply(np.prod([np.divide(char_bag[key][1], hate_count) for key in LT0]), p_hate)


In [0]:
h_numerator  #2.5591876996227343e-06

2.5591876996227335e-06

##Nohate wins!

The nohate probability is larger (or less small from another perspective). So we would predict nohate or 0 for this tweet. And it is in fact labeled as a nohate tweet so we chalk up a correct prediction. Go Bayes!

# Challenge 4

What we need now is the equivalent of knn from module 1. As a reminder, here is its signature line:
<pre>
def knn(target_vector:list, crowd_table:dframe, answer_column:str, k:int, dfunc:Callable) -> int:
</pre>

The signature line needs to change. Here is what I propose:
<pre>
def bayes(evidence:set, evidence_bag:dict, training_table:dframe) -> tuple:
</pre>
where
* evidence is the set of chars we saw in the tweet, e.g., LT0.
* evidence_bag is the dictionary we built from the training table, e.g., char_bag.
* training_table is the training table. You can assume that the training_table has a label column and that it contains integer labels, e.g., 0, 1, etc.

Given just these values, I believe we can compute what we need for Naive Bayes.

Note that I have the function returning a tuple. What I propose is for the function to return the probabilities for all the classes and let someone outside figure out which is the largest. I am doing this because I don't want to lose the probabilities. We may be able to use them in later weeks to shift the outcome in directions that we want, e.g., to overcome bias in our training data.

For testing, try it on LT0 and make sure you get same results as above.

## Important note 1

I would not worry about efficiency here. Write the most understandable algorithm you can. We can worry about making it more efficient later.

In particular, given that you are only passing in the training_table, you will have to compute number of classes, class counts and class probabilities from it. Rather not rely on globals.

## Important note 2

Do not hard-wire your function to the hate-nohate 2-class example. Next week we will be looking at a new problem that has 3 classes. I expect your function to spit out a tuple with 3 probabilities in that case.


In [0]:
def bayes(evidence:set, evidence_bag:dict, training_table:dframe) -> tuple:
  assert isinstance(evidence, set), f'evidence not a set but instead a {type(evidence)}'
  assert isinstance(evidence_bag, dict), f'evidence_bag not a dict but instead a {type(evidence_bag)}'
  assert isinstance(training_table, pd.core.frame.DataFrame), f'training_table not a dataframe but instead a {type(training_table)}'
  assert 'label' in training_table, f'label column is not found in training_table'
  assert training_table.label.dtype == int, f"label must be an int column (possibly wrangled); instead it has type({training_table.label.dtype})"

  #your code here
  label_list = training_table['label'].tolist()

  variation = max(label_list)+1
  
  count = [label_list.count(i) for i in range(variation)]
  p_count = [j/len(training_table) for j in count]

  return tuple(np.multiply(np.prod([np.divide(evidence_bag[key][i], count[i]) for key in evidence]), p_count[i]) for i in range(variation))


In [0]:
bayes(LT0, char_bag, training_table)  #(1.832109487333443e-05, 2.5591876996227335e-06)

(1.832109487333442e-05, 2.5591876996227335e-06)

# Challenge 5

We are now ready for bayes_tester. I'll give you the start of the function below. I would like it to be a little different than knn_tester, which produces a dictionary of the 4 cases. Have bayes_tester return a list of the tuples you get back from bayes. We can mess with the 4 cases a little later.

<pre>
def bayes_tester(testing_table:dframe, evidence_bag:dict, training_table:dframe, parser:Callable) -> list:
</pre>

You noticed I added a parser argument? I am doing this to make our tester a bit more general. This week we are pulling chars out of text. Next week we will be pulling words out of text. The parser function is used to transform the text into the form you want. I'll give you the parser for this week below.

In [0]:
def char_set_builder(text:str) -> list:
  the28 = set(text).intersection(set('abcdefghijklmnopqrstuvwxyz!#'))
  return list(the28)


One more thing. I am going to have the function assume that the text is under a column called 'text'. That means we have to re-wrangle.

In [0]:
bayes_testing_table = testing_table.rename(columns={"tweet": "text"})

Ok, I think you are good to go. Define knn_tester and then test it as follows:
<pre>
%%time
prob_list = bayes_tester(bayes_testing_table, char_bag, training_table, char_set_builder)
</pre>

In [0]:
def bayes_tester(testing_table:dframe, evidence_bag:dict, training_table:dframe, parser:Callable) -> list:
  assert isinstance(testing_table, pd.core.frame.DataFrame), f'test_table not a dataframe but instead a {type(testing_table)}'
  assert isinstance(evidence_bag, dict), f'evidence_bag not a dict but instead a {type(evidence_bag)}'
  assert isinstance(training_table, pd.core.frame.DataFrame), f'training_table not a dataframe but instead a {type(training_table)}'
  assert callable(parser), f'parser not a function but instead a {type(parser)}'
  assert 'label' in training_table, f'label column is not found in training_table'
  assert training_table.label.dtype == int, f"label must be an int column (possibly wrangled); instead it has type({training_table.label.dtype})"
  assert 'text' in testing_table, f'text column is not found in testing_table'

  #your code here
  return [bayes(set(parser(row['text'])), evidence_bag, training_table) for index, row in testing_table.iterrows()]

In [0]:
%%time

prob_list = bayes_tester(bayes_testing_table, char_bag, training_table, char_set_builder)

CPU times: user 11.3 s, sys: 14.3 ms, total: 11.3 s
Wall time: 11.3 s


In [0]:
prob_list[:10]

[(1.832109487333442e-05, 2.5591876996227335e-06),
 (0.15900835851937165, 0.014463929660598234),
 (0.022763656207971168, 0.003299626184482346),
 (0.008792432048100953, 0.001215500810699199),
 (0.013079046652190255, 0.001951518430251379),
 (0.02180312247975622, 0.002475393220562259),
 (0.0267059396671276, 0.00405406478822823),
 (0.13116754249938786, 0.010487708053576284),
 (0.0005928977307361215, 7.236560549700605e-05),
 (0.00041528213972383585, 0.0001015697314194241)]

My results below. Note that it is not unusual for your values to be different at the miniscule level. Make sure they match up to 5 digits.
<pre>
[(1.832109487333443e-05, 2.5591876996227335e-06),
 (0.15900835851937167, 0.014463929660598234),
 (0.02276365620797117, 0.0032996261844823473),
 (0.008792432048100953, 0.001215500810699199),
 (0.013079046652190254, 0.0019515184302513792),
 (0.02180312247975622, 0.0024753932205622588),
 (0.026705939667127598, 0.00405406478822823),
 (0.13116754249938783, 0.010487708053576284),
 (0.0005928977307361218, 7.236560549700605e-05),
 (0.0004152821397238359, 0.00010156973141942408)]
 </pre>

# Challenge 6

Go ahead and compute the confusion dictionary that has the 4 cases.

In [0]:
actuals = testing_table.label.to_list()  #you need the actual labels

In [0]:
#your code here

case_dict = {(0, 0): 0, (0, 1): 0, (1, 0): 0, (1, 1): 0}
index = 0
for i in prob_list:
  node = (0 if i[0] > i[1] else 1, actuals[index])
  for i in case_dict:
    if node == i:
      case_dict[i] += 1
  index += 1

In [0]:
case_dict  #{(0, 0): 9172, (0, 1): 664, (1, 0): 0, (1, 1): 0}

{(0, 0): 9172, (0, 1): 664, (1, 0): 0, (1, 1): 0}

##Epic fail!

I can just look at the values and see that I am always predicting 0. And if I look at the first 10 values in prob_list, this is confirmed. The 0th item is always bigger than the 1st item. Sometimes not by much. But bigger all the same.

My conjecture is that most of the tweets contain similar chars. So chars are not a good way to differentiate. So what is really making the difference is the P(O), i.e., P(hate) and P(nohate). Reminder: P(nohate) is 0.9316321862451075. I conjecture that if the P(O) probabilities were closer, we would get better results.

But seriously, no one uses chars as evidence. It is words that we want. We are set up for that. All we have to do is define a new parser. That is coming up next week :)

##Reminder to update your library with our new functions

